# Generate shape coefficients

The following steps are performed to obtain all the artifacts for shape coefficient use case:

1. Read parameters file
2. Read Mesh description
   * Its path can be specified
   * Or it can be obtained from the simulation configuration file
3. Read data for pressure coefficient

Initialize Path Manager

In [1]:
from cfdmod.use_cases.pressure.path_manager import CePathManager
import pathlib

path_manager = CePathManager(output_path=pathlib.Path("./output/pressure"))

Read post-processing Ce config file

In [6]:
from cfdmod.use_cases.pressure.shape.Ce_config import CeCaseConfig

config_path = pathlib.Path("./fixtures/tests/pressure/Ce_params.yaml")
post_proc_cfg = CeCaseConfig.from_file(config_path)

post_proc_cfg

WindowsPath('fixtures/tests/yaml')

Normalize LNAS Mesh [Example]

In [9]:
from nassu.lnas import LagrangianFormat

original_mesh_path = pathlib.Path("./fixtures/tests/pressure/galpao/galpao.lnas")
original_mesh = LagrangianFormat.from_file(original_mesh_path)

new_verts = original_mesh.geometry.vertices.copy()

new_verts[:, 0] -= new_verts[:, 0].min()
new_verts[:, 1] -= new_verts[:, 1].min()
new_verts[:, 2] -= new_verts[:, 2].min()

original_mesh.geometry.vertices = new_verts

original_mesh.to_file(original_mesh_path.parent / f"{original_mesh.name}.normalized.lnas")

print(
    original_mesh.name,
    original_mesh.geometry.vertices.shape,
    original_mesh.geometry.triangles.shape,
)

original_mesh.geometry.vertices[:, 0].min(), original_mesh.geometry.vertices[
    :, 1
].min(), original_mesh.geometry.vertices[:, 2].min()

galpao (1549, 3) (2915, 3)


(0.0, 0.0, 0.0)

Read Normalized LNAS Mesh

In [10]:
from nassu.lnas import LagrangianFormat

mesh_path = pathlib.Path("./fixtures/tests/pressure/galpao/galpao.normalized.lnas")
mesh = LagrangianFormat.from_file(mesh_path)

mesh.name, mesh.geometry.vertices.shape, mesh.geometry.triangles.shape

('galpao', (1549, 3), (2915, 3))

Read Pressure Coefficient data

In [11]:
import pandas as pd

cp_data_path = pathlib.Path("./fixtures/tests/pressure/data/cp_t.resampled.hdf")
cp_data = pd.read_hdf(cp_data_path)

cp_data_to_use = cp_data.to_frame() if isinstance(cp_data, pd.Series) else cp_data

Add region index to pressure coefficient data

In [12]:
from cfdmod.api.vtk.write_vtk import write_polydata, merge_polydata
from cfdmod.logger import logger
from cfdmod.use_cases.pressure.shape.Ce_data import process_surface, get_surfaces_raw_data
from cfdmod.use_cases.pressure.geometry import get_excluded_surfaces

n_timesteps = cp_data_to_use["time_step"].unique().shape[0]

for cfg_label, cfg in post_proc_cfg.shape_coefficient.items():
    processed_polydata = []
    logger.info(f"Processing {cfg_label} ...")
    
    sfc_dict = {set_lbl: sfc_list for set_lbl, sfc_list in cfg.sets.items()}
    sfc_dict |= {sfc: [sfc] for sfc in mesh.surfaces.keys() if sfc not in cfg.surfaces_in_sets}
    
    surfaces_to_process = get_surfaces_raw_data(surface_dict=sfc_dict, cfg=cfg, mesh=mesh)

    for sfc_lbl, raw_surface in surfaces_to_process.items():
        logger.info(f"Processing surface {sfc_lbl}")
        
        processed_surface = process_surface(
            raw_surface=raw_surface,
            cfg=cfg,
            cp_data=cp_data_to_use,
            n_timesteps=n_timesteps,
        )

        processed_polydata.append(processed_surface.polydata)

        logger.info(f"Processed surface {sfc_lbl}")

    # Output 5: VTK
    merged_polydata = merge_polydata(processed_polydata)
    write_polydata(path_manager.get_vtp_path(mesh.name, cfg_label), merged_polydata)
    
    sfc_list = [sfc for sfc in cfg.zoning.exclude if sfc in mesh.surfaces.keys()]
    
    sfc_list += [sfc for set_lbl, sfc_set in cfg.sets.items() for sfc in sfc_set if set_lbl in cfg.zoning.exclude]
    if len(sfc_list) != 0:
        excluded_sfcs = get_excluded_surfaces(mesh=mesh, sfc_list=sfc_list)
        excluded_sfcs.export_stl(path_manager.get_excluded_surface_path(cfg_label))

    logger.info(f"Merged polydata for {cfg_label}")

[2023-11-01 11:23:55,805] [INFO] - cfdmod - Processing pattern_1 ... (2198819689.py:11)
[2023-11-01 11:23:55,809] [INFO] - cfdmod - Surface p1_xp ignored! (Ce_data.py:85)
[2023-11-01 11:23:55,812] [INFO] - cfdmod - Processing surface lanternim (2198819689.py:19)
[2023-11-01 11:23:56,142] [INFO] - cfdmod - Processed surface lanternim (2198819689.py:30)
[2023-11-01 11:23:56,143] [INFO] - cfdmod - Processing surface m1_yp (2198819689.py:19)
[2023-11-01 11:23:56,195] [INFO] - cfdmod - Processed surface m1_yp (2198819689.py:30)
[2023-11-01 11:23:56,196] [INFO] - cfdmod - Processing surface m2_zp (2198819689.py:19)
[2023-11-01 11:23:56,256] [INFO] - cfdmod - Processed surface m2_zp (2198819689.py:30)
[2023-11-01 11:23:56,259] [INFO] - cfdmod - Processing surface m3_zm (2198819689.py:19)
[2023-11-01 11:23:56,331] [INFO] - cfdmod - Processed surface m3_zm (2198819689.py:30)
[2023-11-01 11:23:56,331] [INFO] - cfdmod - Processing surface p2_xp (2198819689.py:19)
[2023-11-01 11:23:56,375] [WARNIN